In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import product
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from xgboost import plot_importance
import time
import sys
import gc
import pickle

In [2]:
train=pd.read_csv('train.csv',parse_dates=['Datetime'])
test=pd.read_csv('test.csv',parse_dates=['Datetime'])
sub=pd.read_csv('submission.csv')

In [3]:
train.head()

,ID,Datetime,Count
0,0,2012-08-25 00:00:00,8
1,1,2012-08-25 01:00:00,2
2,2,2012-08-25 02:00:00,6
3,3,2012-08-25 03:00:00,2
4,4,2012-08-25 04:00:00,2


In [4]:
test.head()

,ID,Datetime
0,18288,2014-09-26 00:00:00
1,18289,2014-09-26 01:00:00
2,18290,2014-09-26 02:00:00
3,18291,2014-09-26 03:00:00
4,18292,2014-09-26 04:00:00


In [5]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

In [6]:
def create_date_featues(df):

    df['Year'] = pd.to_datetime(df['Datetime']).dt.year

    df['Month'] = pd.to_datetime(df['Datetime']).dt.month

    df['Day'] = pd.to_datetime(df['Datetime']).dt.day

    df['Dayofweek'] = pd.to_datetime(df['Datetime']).dt.dayofweek

    df['DayOfyear'] = pd.to_datetime(df['Datetime']).dt.dayofyear

    df['Week'] = pd.to_datetime(df['Datetime']).dt.week

    df['Quarter'] = pd.to_datetime(df['Datetime']).dt.quarter 
    df['Semester'] = np.where(df['Quarter'].isin([1,2]),1,2)

    df['Is_weekend'] = np.where(df['Dayofweek'].isin([5,6]),1,0)

    df['Is_weekday'] = np.where(df['Dayofweek'].isin([0,1,2,3,4]),1,0)
    df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour

    return df

In [7]:
df=create_date_featues(df)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


,ID,Datetime,Count,train_or_test,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Semester,Is_weekend,Is_weekday,Hour
0,0,2012-08-25 00:00:00,8.0,train,2012,8,25,5,238,34,3,2,1,0,0
1,1,2012-08-25 01:00:00,2.0,train,2012,8,25,5,238,34,3,2,1,0,1
2,2,2012-08-25 02:00:00,6.0,train,2012,8,25,5,238,34,3,2,1,0,2
3,3,2012-08-25 03:00:00,2.0,train,2012,8,25,5,238,34,3,2,1,0,3
4,4,2012-08-25 04:00:00,2.0,train,2012,8,25,5,238,34,3,2,1,0,4


In [8]:
def create_count_agg_monthwise_features(df, gpby_cols, target_col, agg_funcs):
    '''
    Creates various "count" agg features with given agg functions
    Parameters
    --------
        df : dataframe 
        The dataframe to calculate the value counts for.
        gpby_cols : 
            this variable used for grouping and aggregating
        
    '''

    gpby = df.groupby(gpby_cols)
    newdf = df[gpby_cols].drop_duplicates().reset_index(drop=True)
    for agg_name, agg_func in agg_funcs.items():
        aggdf = gpby[target_col].agg(agg_func).reset_index()
        aggdf.rename(columns={target_col:target_col+'_'+agg_name}, inplace=True)
        newdf = newdf.merge(aggdf, on=gpby_cols, how='left')
    return newdf

In [9]:
agg_df = create_count_agg_monthwise_features(df.loc[df.train_or_test=='train', :], 
                                              gpby_cols=['Hour'], 
                                              target_col='Count', 
                                              agg_funcs={'mean':np.mean, 
                                              'median':np.median, 'max':np.max, 
                                              'min':np.min, 'std':np.std})

In [10]:
agg_df

,Hour,Count_mean,Count_median,Count_max,Count_min,Count_std
0,0,147.695538,108.0,686.0,2.0,139.160659
1,1,117.868766,84.0,596.0,2.0,111.824076
2,2,99.753281,70.0,580.0,2.0,98.282930
3,3,82.422572,56.0,604.0,2.0,82.079253
4,4,69.375328,46.0,458.0,2.0,69.777030
5,5,62.102362,42.0,412.0,2.0,63.730172
6,6,66.776903,45.0,646.0,2.0,71.147420
7,7,77.443570,52.0,498.0,2.0,82.727519
8,8,89.181102,62.0,558.0,2.0,91.214445
9,9,115.241470,74.0,696.0,2.0,119.304366


In [11]:
df=df.merge(agg_df,on=['Hour'],how='left')
df.head()

,ID,Datetime,Count,train_or_test,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Semester,Is_weekend,Is_weekday,Hour,Count_mean,Count_median,Count_max,Count_min,Count_std
0,0,2012-08-25 00:00:00,8.0,train,2012,8,25,5,238,34,3,2,1,0,0,147.695538,108.0,686.0,2.0,139.160659
1,1,2012-08-25 01:00:00,2.0,train,2012,8,25,5,238,34,3,2,1,0,1,117.868766,84.0,596.0,2.0,111.824076
2,2,2012-08-25 02:00:00,6.0,train,2012,8,25,5,238,34,3,2,1,0,2,99.753281,70.0,580.0,2.0,98.282930
3,3,2012-08-25 03:00:00,2.0,train,2012,8,25,5,238,34,3,2,1,0,3,82.422572,56.0,604.0,2.0,82.079253
4,4,2012-08-25 04:00:00,2.0,train,2012,8,25,5,238,34,3,2,1,0,4,69.375328,46.0,458.0,2.0,69.777030


In [12]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test','Datetime'},axis=1,inplace=True)
test.drop(columns={'train_or_test','Datetime'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
train['Count']=np.log1p(train['Count'])
train.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Count,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Semester,Is_weekend,Is_weekday,Hour,Count_mean,Count_median,Count_max,Count_min,Count_std
0,0,2.197225,2012,8,25,5,238,34,3,2,1,0,0,147.695538,108.0,686.0,2.0,139.160659
1,1,1.098612,2012,8,25,5,238,34,3,2,1,0,1,117.868766,84.0,596.0,2.0,111.824076
2,2,1.945910,2012,8,25,5,238,34,3,2,1,0,2,99.753281,70.0,580.0,2.0,98.282930
3,3,1.098612,2012,8,25,5,238,34,3,2,1,0,3,82.422572,56.0,604.0,2.0,82.079253
4,4,1.098612,2012,8,25,5,238,34,3,2,1,0,4,69.375328,46.0,458.0,2.0,69.777030


In [14]:
train.columns

Index(['ID', 'Count', 'Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear', 'Week',
       'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour', 'Count_mean',
       'Count_median', 'Count_max', 'Count_min', 'Count_std'],
      dtype='object')

## TRAIN AND TEST DATA SPLIT

In [15]:
x_train=train.loc[:,['Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear', 'Week',
       'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour', 'Count_mean',
       'Count_median', 'Count_max', 'Count_min', 'Count_std']]
y_train=train.loc[:,['Count']]
test=test.loc[:,['Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear', 'Week',
       'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour', 'Count_mean',
       'Count_median', 'Count_max', 'Count_min', 'Count_std']]

## FEATURE SCALING

In [16]:
col=x_train.columns
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
st.fit(x_train)
x_train=st.transform(x_train)
test=st.transform(test)

In [17]:
x_train=pd.DataFrame(x_train,columns=col)
x_train1=x_train.copy()
x_train[x_train<0]=0
x_train1[x_train1>0]=0
x_train1=x_train1.add_suffix('_neg')
x_train=x_train.join(x_train1)

In [18]:
x_train

,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Semester,Is_weekend,Is_weekday,Hour,Count_mean,Count_median,Count_max,Count_min,Count_std,Year_neg,Month_neg,Day_neg,Dayofweek_neg,DayOfyear_neg,Week_neg,Quarter_neg,Semester_neg,Is_weekend_neg,Is_weekday_neg,Hour_neg,Count_mean_neg,Count_median_neg,Count_max_neg,Count_min_neg,Count_std_neg
0,0.000000,0.414787,1.052537,0.999509,0.506823,0.483704,0.432369,0.966442,1.579688,0.000000,0.000000,0.203954,0.441489,0.000000,0.0,0.000000,-1.693104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.579688,-1.661325,0.000000,0.000000,-0.593941,0.0,-0.035308
1,0.000000,0.414787,1.052537,0.999509,0.506823,0.483704,0.432369,0.966442,1.579688,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-1.693104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.579688,-1.516862,-0.492280,-0.370963,-0.964725,0.0,-0.655481
2,0.000000,0.414787,1.052537,0.999509,0.506823,0.483704,0.432369,0.966442,1.579688,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-1.693104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.579688,-1.372399,-0.915143,-0.844894,-1.030642,0.0,-0.962683
3,0.000000,0.414787,1.052537,0.999509,0.506823,0.483704,0.432369,0.966442,1.579688,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-1.693104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.579688,-1.227936,-1.319686,-1.318824,-0.931767,0.0,-1.330289
4,0.000000,0.414787,1.052537,0.999509,0.506823,0.483704,0.432369,0.966442,1.579688,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-1.693104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.579688,-1.083473,-1.624243,-1.657346,-1.533261,0.0,-1.609384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18283,1.170703,0.707172,1.052537,0.000655,0.793230,0.818901,0.432369,0.966442,0.000000,0.633037,1.083473,1.086745,0.983124,1.433012,0.0,1.015982,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.633037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
18284,1.170703,0.707172,1.052537,0.000655,0.793230,0.818901,0.432369,0.966442,0.000000,0.633037,1.227936,0.917711,1.118532,0.732642,0.0,0.742147,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.633037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
18285,1.170703,0.707172,1.052537,0.000655,0.793230,0.818901,0.432369,0.966442,0.000000,0.633037,1.372399,0.640540,0.780011,0.436015,0.0,0.457450,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.633037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
18286,1.170703,0.707172,1.052537,0.000655,0.793230,0.818901,0.432369,0.966442,0.000000,0.633037,1.516862,0.654632,0.983124,0.073470,0.0,0.362490,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.633037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [19]:
test=pd.DataFrame(test,columns=col)
test1=test.copy()
test[test<0]=0
test1[test1>0]=0
test1=test1.add_suffix('_neg')
test=test.join(test1)

## XGBOOST REGRESSOR MODEL

In [20]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    x_train, 
    y_train)

[06:52:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=300, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.8, verbosity=1)

In [21]:
pred=model.predict(test)

In [22]:
len(test)

5112

In [23]:
sub['Count']=np.expm1(pred)

In [25]:
model.score(x_train, y_train)

0.9575371080662516

In [26]:
sub.to_csv('sub1.csv',index=False)

In [27]:
sub

,ID,Count
0,18288,597.013489
1,18289,444.266449
2,18290,353.288300
3,18291,298.224396
4,18292,248.857849
...,...,...
5107,23395,235.153793
5108,23396,229.743469
5109,23397,238.766663
5110,23398,254.081131
